In [ ]:
!pip install ebaysdk


     |████████████████████████████████| 81kB 2.6MB/s 


In [ ]:
import ebaysdk
from ebaysdk.trading import Connection
import pandas as pd

In [ ]:

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#Getting Data

In [ ]:
#Get CSV file
df = pd.read_csv("/content/drive/My Drive/Bedding/Data Sets/Test Inventory.csv")
df

,Brand,Type,Size,Thread Count,Color,Material,Quantity,Start Price,Condition ID,Category ID,Description,Picture URL
0,Wamsutta DreamZone,Pillowcase Set,King,750,Charcoal,Cotton,3,40,1000,43397,"Includes 2 King Pillowcases (20 in x 40 in), m...",NaN
1,Wamsutta DreamZone,Pillowcase Set,King,750,Spice Red,Cotton,2,40,1000,43397,"Includes 2 King Pillowcases (20 in x 40 in), m...",NaN
2,Wamsutta DreamZone,Pillowcase Set,King,750,Aqua,Cotton,2,40,1000,43397,"Includes 2 King Pillowcases (20 in x 40 in), m...",NaN
3,Wamsutta DreamZone,Pillowcase Set,King,750,Sky,Cotton,1,40,1000,43397,"Includes 2 King Pillowcases (20 in x 40 in), m...",NaN
4,Wamsutta DreamZone,Pillowcase Set,King,750,Ivory,Cotton,1,40,1000,43397,"Includes 2 King Pillowcases (20 in x 40 in), m...",NaN
5,Wamsutta DreamZone,Pillowcase Set,King,750,Laven,Cotton,1,40,1000,43397,"Includes 2 King Pillowcases (20 in x 40 in), m...",NaN
6,Wamsutta DreamZone,Pillowcase Set,Queen,750,Laven,Cotton,2,40,1000,43397,"Includes 2 Queen Pillowcases (20 in x 32 in), ...",NaN


#Process Data into Item Object

In [ ]:
#Create Item class, and store all of them in an inventory array
inventory = []

In [ ]:
#Item class 
class Item:
  def __init__(self, brand, Type, size,
               threadCount, color, material, startPrice, 
               condition, category, description, pictures):
    
    self.name = self.makeName(brand, Type, size, threadCount, color)
    self.color = color.title()
    self.brand = brand.title()
    self.size = size.title()
    self.price = startPrice
    self.Type = Type.title()
    self.material = material.title()
    self.condition = str(condition)
    self.category = str(category)
    self.description = description

  def makeName(self, brand, Type, size, threadCount, color):
    if threadCount == "?":
      return "New {0} {1} {2} {3} Size".format(color.title(), brand.title(), Type.title(), size.title()) 
    else:
      return "New {0} {1} {2} {3} Size {4} TC".format(color.title(), brand.title(), Type.title(), size.title(), threadCount) 

In [ ]:
for index, row in df.iterrows():
    temp = Item(row["Brand"], row["Type"], row["Size"], 
                row["Thread Count"], row["Color"], row["Material"], row["Start Price"],
                row["Condition ID"], row["Category ID"], row["Description"],
                row["Picture URL"])
    inventory.append(temp)


In [ ]:
x = inventory[0]
print(x.name, x.price, x.condition, x.category, x.description )
print(len(inventory))

New Charcoal Wamsutta Dreamzone Pillowcase Set King Size 750 TC 40 1000 43397 Includes 2 King Pillowcases (20 in x 40 in), made from 100% Cotton
7


#Post all Items on Ebay using API

In [ ]:
api = Connection(config_file="/content/drive/My Drive/Bedding/Data Sets/ebay.yaml", domain="api.ebay.com", debug=True)
api

In [ ]:
for item in inventory:
  request = {
        "Item": {
            "Title": item.name,
            "Country": "US",
            "Location": "CA",
            "Site": "US",
            "ConditionID": item.condition,
            "PaymentMethods": "PayPal",
            "PayPalEmailAddress": "alexm7701@gmail.com",
            "PrimaryCategory": {"CategoryID": item.category},
            "ListingDuration" : "Days_10",
            "Description": item.description,
            "StartPrice": item.price,
            "Currency": "USD",
            "PictureDetails": {
                "PictureURL": []},
            "ItemSpecifics": {
                "NameValueList" :[
                    {
                    "Name": "Brand",
                    "Value": item.brand
                    },
                    {
                    "Name": "Color",
                    "Value": item.color
                    },
                    {
                    "Name": "Size",
                    "Value": item.size
                    },
                    {
                    "Name": "Material",
                    "Value": item.material
                    },
                    {
                    "Name": "Type",
                    "Value": item.Type
                    }
                ]
            },
            "ReturnPolicy": {
                "ReturnsAcceptedOption": "ReturnsNotAccepted"
            },
            "ShippingDetails": {
                "ShippingServiceOptions": {
                    "FreeShipping": "True",
                    "ShippingService": "USPSFirstClass"
                }
            },
            "DispatchTimeMax": "1"
        }
    }
  
  api.execute("AddFixedPriceItem", request)

2020-09-01 23:35:28,524 ebaysdk [DEBUG]:execute: verb=AddFixedPriceItem data={'Item': {'Title': 'New Charcoal Wamsutta Dreamzone Pillowcase Set King Size 750 TC', 'Country': 'US', 'Location': 'CA', 'Site': 'US', 'ConditionID': '1000', 'PaymentMethods': 'PayPal', 'PayPalEmailAddress': 'alexm7701@gmail.com', 'PrimaryCategory': {'CategoryID': '43397'}, 'ListingDuration': 'Days_10', 'Description': 'Includes 2 King Pillowcases (20 in x 40 in), made from 100% Cotton', 'StartPrice': 40, 'Currency': 'USD', 'PictureDetails': {'PictureURL': []}, 'ItemSpecifics': {'NameValueList': [{'Name': 'Brand', 'Value': 'Wamsutta Dreamzone'}, {'Name': 'Color', 'Value': 'Charcoal'}, {'Name': 'Size', 'Value': 'King'}, {'Name': 'Material', 'Value': 'Cotton'}, {'Name': 'Type', 'Value': 'Pillowcase Set'}]}, 'ReturnPolicy': {'ReturnsAcceptedOption': 'ReturnsNotAccepted'}, 'ShippingDetails': {'ShippingServiceOptions': {'FreeShipping': 'True', 'ShippingService': 'USPSFirstClass'}}, 'DispatchTimeMax': '1'}}
2020-09-0